In [40]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [3]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [4]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [5]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [19]:
call_start_logs = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="call_start_logs", properties=jdbc_properties))

call_end_logs = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="call_end_logs", properties=jdbc_properties))

In [20]:
call_start_logs.show()

+------------+-------------------+
|phone_number|         start_time|
+------------+-------------------+
|         PN1|2022-01-01 10:20:00|
|         PN1|2022-01-01 16:25:00|
|         PN2|2022-01-01 12:30:00|
|         PN3|2022-01-02 10:00:00|
|         PN3|2022-01-02 12:30:00|
|         PN3|2022-01-03 09:20:00|
+------------+-------------------+



In [21]:
call_end_logs.show()

+------------+-------------------+
|phone_number|           end_time|
+------------+-------------------+
|         PN1|2022-01-01 10:45:00|
|         PN1|2022-01-01 17:05:00|
|         PN2|2022-01-01 12:55:00|
|         PN3|2022-01-02 10:20:00|
|         PN3|2022-01-02 12:50:00|
|         PN3|2022-01-03 09:40:00|
+------------+-------------------+



In [22]:
window = Window.partitionBy("phone_number").orderBy("start_time")
call_start_logs = call_start_logs.withColumn("row_num1", row_number().over(window))

In [23]:
window = Window.partitionBy("phone_number").orderBy("end_time")
call_end_logs = call_end_logs.withColumn("row_num2", row_number().over(window))

In [28]:
join_condition = (call_start_logs["phone_number"] == call_end_logs["phone_number"]) \
                 & (call_start_logs["row_num1"] == call_end_logs["row_num2"])

In [34]:
join_df = call_start_logs.join(call_end_logs, join_condition, 'inner')

In [37]:
join_df.show()

+------------+-------------------+--------+------------+-------------------+--------+
|phone_number|         start_time|row_num1|phone_number|           end_time|row_num2|
+------------+-------------------+--------+------------+-------------------+--------+
|         PN1|2022-01-01 10:20:00|       1|         PN1|2022-01-01 10:45:00|       1|
|         PN1|2022-01-01 16:25:00|       2|         PN1|2022-01-01 17:05:00|       2|
|         PN2|2022-01-01 12:30:00|       1|         PN2|2022-01-01 12:55:00|       1|
|         PN3|2022-01-02 10:00:00|       1|         PN3|2022-01-02 10:20:00|       1|
|         PN3|2022-01-02 12:30:00|       2|         PN3|2022-01-02 12:50:00|       2|
|         PN3|2022-01-03 09:20:00|       3|         PN3|2022-01-03 09:40:00|       3|
+------------+-------------------+--------+------------+-------------------+--------+



In [48]:
join_df.withColumn("time_diff_minutes", abs(col("start_time").cast('long') - col("end_time").cast('long'))/60).show(truncate = False)

+------------+-------------------+--------+------------+-------------------+--------+-----------------+
|phone_number|start_time         |row_num1|phone_number|end_time           |row_num2|time_diff_minutes|
+------------+-------------------+--------+------------+-------------------+--------+-----------------+
|PN1         |2022-01-01 10:20:00|1       |PN1         |2022-01-01 10:45:00|1       |25.0             |
|PN1         |2022-01-01 16:25:00|2       |PN1         |2022-01-01 17:05:00|2       |40.0             |
|PN2         |2022-01-01 12:30:00|1       |PN2         |2022-01-01 12:55:00|1       |25.0             |
|PN3         |2022-01-02 10:00:00|1       |PN3         |2022-01-02 10:20:00|1       |20.0             |
|PN3         |2022-01-02 12:30:00|2       |PN3         |2022-01-02 12:50:00|2       |20.0             |
|PN3         |2022-01-03 09:20:00|3       |PN3         |2022-01-03 09:40:00|3       |20.0             |
+------------+-------------------+--------+------------+--------

In [58]:
time_df = join_df.withColumn("time_diff_minutes", expr("abs(cast(start_time as long) - cast(end_time as long))/60"))

In [63]:
time_df.drop(*["row_num1", "row_num2", call_start_logs["phone_number"]]).show()

+-------------------+------------+-------------------+-----------------+
|         start_time|phone_number|           end_time|time_diff_minutes|
+-------------------+------------+-------------------+-----------------+
|2022-01-01 10:20:00|         PN1|2022-01-01 10:45:00|             25.0|
|2022-01-01 16:25:00|         PN1|2022-01-01 17:05:00|             40.0|
|2022-01-01 12:30:00|         PN2|2022-01-01 12:55:00|             25.0|
|2022-01-02 10:00:00|         PN3|2022-01-02 10:20:00|             20.0|
|2022-01-02 12:30:00|         PN3|2022-01-02 12:50:00|             20.0|
|2022-01-03 09:20:00|         PN3|2022-01-03 09:40:00|             20.0|
+-------------------+------------+-------------------+-----------------+

